bouncing_ball_nonsmooth.ipynb
------------------------
Bouncing ball in 1D (vertical) with unilateral contact enforced by a
Lagrange multiplier (impulse) λ ≥ 0 and a velocity-level restitution constraint:

- Ball: mass m, radius r
- Ground: horizontal plane at u = 0
- Contact: restitution coefficient e
- Gravity acts downward (negative y direction)

$
\begin{cases}
    ma_{n+1} = -mg\\
    mw_{n+1} = \lambda_{n+1}\\
    0\leq\lambda_{n+1}\perp v_{n+1} + ev_n \geq 0\\
    \tilde u_{n+1} = u_n+\Delta t v_n+\frac{1}{2}\Delta t^2 a_n\\
    \tilde v_{n+1} = v_n + \frac{\Delta t}{2}(a_n + a_{n+1})\\
    v_{n+1} = \tilde v_{n+1}+w_{n+1}\\
    u_{n+1} = \tilde u_{n+1} + \frac{\Delta t}{2} w_{n+1}
\end{cases}
$

How to solve
- compute smooth displacement prediction $\tilde u_{n+1}$ assuming free flight
- compute forward velocity prediction $v^*_{n+1} = v_n + \Delta t a_n$
- compute the gap $g=\tilde u_{n+1}$ and its rate $\dot g=v^*_{n+1}$
    - if both are negative (contact): 
        - set $v_{n+1}=-ev_n$
        - compute $a_{n+1}=-g$
        - compute $\tilde v_{n+1}$
        - compute $w_{n+1}$ and $\lambda_{n+1}$
        - update displacement and velocity $u_{n+1}$, $v_{n+1}$
    - if not both are negative (free flight):
        - set $\lambda_{n+1}=w_{n+1}=0$
        - compute $a_{n+1}=-g$
        - compute $\tilde v_{n+1}$
        - update displacement and velocity $u_{n+1}$, $v_{n+1}$


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Tuple
import numpy as np
import plotly.graph_objects as go

In [ ]:
@dataclass
class Params:
    m: float = 1.0            # mass [kg]
    r: float = 0.1            # radius [m] (not explicitly used since we integrate the gap u)
    e: float = 0.9            # restitution coefficient in [0,1]
    g_const: float = 9.81     # gravity acceleration [m/s^2]
    dt: float = 1e-3          # time step [s]
    t_end: float = 2.0        # end time [s]


@dataclass
class State:
    u: float                  # gap to ground [m]
    v: float                  # velocity [m/s] (positive upward)
    a: float                  # acceleration [m/s^2]
    w: float = 0.0            # velocity correction from contact [m/s]
    lam: float = 0.0          # Lagrange multiplier (impulse) [N·s]/[s]=[N]? Here m*w so [kg m/s^2]? We treat as m*w (per-step impulse rate)
    t: float = 0.0            # time [s]


@dataclass
class History:
    # kinematics + contact flags
    t: List[float] = field(default_factory=list)
    u: List[float] = field(default_factory=list)
    v: List[float] = field(default_factory=list)
    a: List[float] = field(default_factory=list)
    w: List[float] = field(default_factory=list)
    lam: List[float] = field(default_factory=list)
    contact: List[bool] = field(default_factory=list)
    # energies
    ekin: List[float] = field(default_factory=list)
    epot: List[float] = field(default_factory=list)
    emech: List[float] = field(default_factory=list)
    edis_inc: List[float] = field(default_factory=list)
    edis: List[float] = field(default_factory=list)
    ealg: List[float] = field(default_factory=list)
    etot: List[float] = field(default_factory=list)
    etot_alg: List[float] = field(default_factory=list)

    def append(self, s: State, is_contact: bool,
               ekin: float, epot: float, emech: float,
               edis_inc: float, edis: float, ealg: float,
               etot: float, etot_alg: float) -> None:
        # state
        self.t.append(s.t)
        self.u.append(s.u)
        self.v.append(s.v)
        self.a.append(s.a)
        self.w.append(s.w)
        self.lam.append(s.lam)
        self.contact.append(is_contact)
        # energies
        self.ekin.append(ekin)
        self.epot.append(epot)
        self.emech.append(emech)
        self.edis_inc.append(edis_inc)
        self.edis.append(edis)
        self.ealg.append(ealg)
        self.etot.append(etot)
        self.etot_alg.append(etot_alg)

    def as_arrays(self) -> Dict[str, np.ndarray]:
        return {k: np.asarray(v) for k, v in self.__dict__.items()}

In [ ]:
class BouncingBallLagrange:
    """
    Time integrator implementing unilateral contact via Lagrange multiplier
    with velocity-level restitution enforcement. It uses the explicit
    nonsmooth Newmark integration scheme.
    """
    def __init__(self, params: Params, u0: float, v0: float):
        self.p = params
        a0 = -self.p.g_const  # upward positive
        self.state = State(u=u0, v=v0, a=a0, w=0.0, lam=0.0, t=0.0)
        self.hist = History()
        # initial energies at n = 0
        ekin0 = 0.5 * self.p.m * v0**2
        epot0 = self.p.m * self.p.g_const * u0
        emech0 = ekin0 + epot0
        edis0 = 0.0
        ealg0 = emech0 - (self.p.dt**2) * 0.125 * self.p.m * (a0**2)
        etot0 = emech0 + edis0
        etot_alg0 = ealg0 + edis0
        self.hist.append(self.state, is_contact=False,
                         ekin=ekin0, epot=epot0, emech=emech0,
                         edis_inc=0.0, edis=edis0, ealg=ealg0,
                         etot=etot0, etot_alg=etot_alg0)

    def step(self) -> Tuple[State, bool]:
        p = self.p
        s = self.state

        # 1) Smooth displacement prediction (free flight)
        u_tilde = s.u + p.dt * s.v + 0.5 * p.dt**2 * s.a

        # 2) Forward velocity prediction (gap rate)
        v_star = s.v + p.dt * s.a

        # 3) Gap and rate
        g = u_tilde
        gdot = v_star

        # 4) Next acceleration from gravity
        a_next = -p.g_const

        # 5) Smooth velocity prediction (trapezoidal)
        v_tilde = s.v + 0.5 * p.dt * (s.a + a_next)

        # 6) Contact detection
        is_contact = (g < 0.0) and (gdot < 0.0)

        if is_contact:
            # Enforce restitution at velocity level
            v_next = -p.e * s.v
            # Correction and impulse
            w_next = v_next - v_tilde
            lam_next = p.m * w_next
            # Displacement update with correction
            u_next = u_tilde + 0.5 * p.dt * w_next
        else:
            w_next = 0.0
            lam_next = 0.0
            v_next = v_tilde
            u_next = u_tilde

        # Advance state n -> n+1
        s_next = State(u=u_next, v=v_next, a=a_next, w=w_next, lam=lam_next, t=s.t + p.dt)
        self.state = s_next

        # ---- Energies at n+1 ----
        edis_inc = - lam_next * 0.5 * (s.v + v_next) if is_contact else 0.0
        edis_prev = self.hist.edis[-1] if len(self.hist.edis) > 0 else 0.0
        edis = edis_prev + edis_inc

        ekin = 0.5 * p.m * (v_next**2)
        epot = p.m * p.g_const * u_next
        emech = ekin + epot
        ealg = emech - (p.dt**2) * 0.125 * p.m * (a_next**2)
        etot = emech + edis
        etot_alg = ealg + edis

        # Store
        self.hist.append(
            s_next, is_contact,
            ekin=ekin, epot=epot, emech=emech,
            edis_inc=edis_inc, edis=edis, ealg=ealg,
            etot=etot, etot_alg=etot_alg
        )
        return s_next, is_contact

    def run(self) -> History:
        n_steps = int(np.ceil(self.p.t_end / self.p.dt))
        for _ in range(n_steps):
            self.step()
        return self.hist


def plot_history(hist: History) -> None:
    H = hist.as_arrays()
    t = H["t"]

    # u(t) with contact markers
    fig_u = go.Figure()
    fig_u.add_trace(go.Scatter(x=t, y=H["u"], mode="lines", name="u (gap) [m]"))
    mask = H["contact"].astype(bool)
    if mask.any():
        fig_u.add_trace(go.Scatter(x=t[mask], y=H["u"][mask], mode="markers",
                                   name="contact", marker=dict(symbol="circle-open")))
    fig_u.update_layout(title="Bouncing ball — gap to ground", xaxis_title="t [s]", yaxis_title="u [m]",
                        xaxis=dict(exponentformat="power"), yaxis=dict(exponentformat="power"))
    fig_u.show()

    # v(t)
    fig_v = go.Figure()
    fig_v.add_trace(go.Scatter(x=t, y=H["v"], mode="lines", name="v [m/s]"))
    fig_v.update_layout(title="Velocity", xaxis_title="t [s]", yaxis_title="v [m/s]",
                        xaxis=dict(exponentformat="power"), yaxis=dict(exponentformat="power"))
    fig_v.show()

    # lambda(t)
    fig_l = go.Figure()
    fig_l.add_trace(go.Scatter(x=t, y=H["lam"], mode="lines", name="λ"))
    fig_l.update_layout(title="Contact impulse-like (λ = m w)", xaxis_title="t [s]", yaxis_title="λ",
                       xaxis=dict(exponentformat="power"), yaxis=dict(exponentformat="power"))
    fig_l.show()

    # Energy balance: ekin, epot, edis
    fig_ebal = go.Figure()
    fig_ebal.add_trace(go.Scatter(x=t, y=H["ekin"], mode="lines", name="E_kin"))
    fig_ebal.add_trace(go.Scatter(x=t, y=H["epot"], mode="lines", name="E_pot"))
    fig_ebal.add_trace(go.Scatter(x=t, y=H["edis"], mode="lines", name="E_dis"))
    fig_ebal.add_trace(go.Scatter(x=t, y=H["emech"], mode="lines", name="E_mech", line=dict(color="black", dash="dot")))
    fig_ebal.update_layout(title="Energy balance", xaxis_title="t [s]", yaxis_title="Energy [J]",
                           xaxis=dict(exponentformat="power"), yaxis=dict(exponentformat="power"))
    fig_ebal.show()
   
    # Energies: emech, etot, ealg, etot_alg
    fig_e = go.Figure()
    fig_e.add_trace(go.Scatter(x=t, y=H["etot"]-H["etot"][0], mode="lines", name="E_total", line=dict(color="black")))
    fig_e.add_trace(go.Scatter(x=t, y=H["etot_alg"]-H["etot_alg"][0], mode="lines", name="E_total_algo", line=dict(color="gray")))
    fig_e.update_layout(title="Total energy variation", xaxis_title="t [s]", yaxis_title="Energy variation [J]",
                       xaxis=dict(exponentformat="power"), yaxis=dict(exponentformat="power"))
    fig_e.show()



### Run the simulation

In [ ]:
# Example initial conditions
params = Params(m=1.0, r=0.0, e=0.9, g_const=9.81, dt=1e-2, t_end=10)
# u is gap (distance between ball bottom and ground). If center at y0, u0 = y0 - r.
u0 = 1.0  # [m]
v0 = 0.0  # [m/s]

sim = BouncingBallLagrange(params, u0=u0, v0=v0)
hist = sim.run()


plot_history(hist)